<a href="https://colab.research.google.com/github/huongtn1112/BTL_ML_Re-Id/blob/main/OSNetModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup


In [1]:
# cd to your preferred directory and clone this repo
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create environment
%cd deep-person-reid/

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (840/840), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 9809 (delta 761), reused 733 (delta 733), pack-reused 8969
Receiving objects: 100% (9809/9809), 9.48 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (7280/7280), done.
/content/deep-person-reid
     |████████████████████████████████| 5.8 MB 4.1 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 185 kB 52.9 MB/s 
     |████████████████████████████████| 42 kB 906 kB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 73 kB 1.8 MB/s 
     |████████████████████████████████| 41 kB 558 kB/s 
     |████████████████████████████████| 68 kB 7.1 MB/s 
/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  

In [2]:
import torchreid

#Load data

In [3]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3487 KB/s, 42 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




#Model osnet_ain_x1_0

In [4]:
model = torchreid.models.build_model(
    name='osnet_ain_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: /root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 124MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [5]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [8]:
engine.run(
    save_dir='log/osnet_ain_x1_0',
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.499 (0.562)	data 0.001 (0.043)	eta 0:18:50	loss 1.1152 (1.1553)	acc 100.0000 (98.4375)	lr 0.000300
epoch: [1/5][20/404]	time 0.480 (0.526)	data 0.001 (0.022)	eta 0:17:31	loss 1.1622 (1.1606)	acc 100.0000 (99.0625)	lr 0.000300
epoch: [1/5][30/404]	time 0.488 (0.515)	data 0.001 (0.015)	eta 0:17:03	loss 1.1296 (1.1550)	acc 100.0000 (99.3750)	lr 0.000300
epoch: [1/5][40/404]	time 0.497 (0.509)	data 0.001 (0.011)	eta 0:16:46	loss 1.1094 (1.1501)	acc 100.0000 (99.5312)	lr 0.000300
epoch: [1/5][50/404]	time 0.482 (0.504)	data 0.001 (0.009)	eta 0:16:33	loss 1.1551 (1.1510)	acc 96.8750 (99.3750)	lr 0.000300
epoch: [1/5][60/404]	time 0.486 (0.501)	data 0.000 (0.008)	eta 0:16:22	loss 1.1843 (1.1521)	acc 100.0000 (99.4271)	lr 0.000300
epoch: [1/5][70/404]	time 0.485 (0.499)	data 0.001 (0.007)	eta 0:16:13	loss 1.1337 (1.1556)	acc 100.0000 (99.4643)	lr 0.000300
epoch: [1/5][80/404]	time 0.486 (0.498)	data 0.001 (0.006)	eta 0:16:05	loss 1.1774 (1.1569)	acc 100.0000 (99.414

#Model osnet_ibn_x1_0

In [9]:
model = torchreid.models.build_model(
    name='osnet_ibn_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


Downloading...
From: https://drive.google.com/uc?id=1sr90V6irlYYDd4_4ISU2iruoRG8J__6l
To: /root/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 67.8MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [10]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)


In [12]:
engine.run(
    save_dir='log/osnet_ibn_x1_0',
    max_epoch=5
    eval_freq=10,
    print_freq=10,
    test_only=False
)


=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.496 (0.575)	data 0.000 (0.040)	eta 0:19:16	loss 4.2562 (4.4828)	acc 25.0000 (29.6875)	lr 0.000300
epoch: [1/5][20/404]	time 0.492 (0.534)	data 0.000 (0.021)	eta 0:17:48	loss 4.5098 (4.4403)	acc 25.0000 (30.3125)	lr 0.000300
epoch: [1/5][30/404]	time 0.490 (0.521)	data 0.001 (0.014)	eta 0:17:15	loss 4.0817 (4.4005)	acc 37.5000 (30.6250)	lr 0.000300
epoch: [1/5][40/404]	time 0.498 (0.514)	data 0.001 (0.011)	eta 0:16:56	loss 4.3038 (4.3692)	acc 31.2500 (31.2500)	lr 0.000300
epoch: [1/5][50/404]	time 0.490 (0.509)	data 0.000 (0.009)	eta 0:16:42	loss 4.1864 (4.3465)	acc 31.2500 (31.5000)	lr 0.000300
epoch: [1/5][60/404]	time 0.511 (0.507)	data 0.000 (0.007)	eta 0:16:34	loss 4.1070 (4.2951)	acc 40.6250 (33.0208)	lr 0.000300
epoch: [1/5][70/404]	time 0.489 (0.505)	data 0.000 (0.007)	eta 0:16:25	loss 4.2282 (4.2755)	acc 37.5000 (33.3929)	lr 0.000300
epoch: [1/5][80/404]	time 0.499 (0.504)	data 0.009 (0.006)	eta 0:16:17	loss 3.7340 (4.2173)	acc 46.8750 (34.4531)	lr 0

#osnet_x1_0

In [13]:
model = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 117MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [14]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [18]:
engine.run(
    save_dir='log/osnet_ibn_x1_0',
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    test_only=False
)


=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.497 (0.547)	data 0.001 (0.043)	eta 0:18:19	loss 6.6038 (6.6328)	acc 3.1250 (0.9375)	lr 0.000300
epoch: [1/5][20/404]	time 0.483 (0.512)	data 0.002 (0.022)	eta 0:17:04	loss 6.5114 (6.6257)	acc 3.1250 (1.0938)	lr 0.000300
epoch: [1/5][30/404]	time 0.476 (0.501)	data 0.001 (0.015)	eta 0:16:36	loss 6.7029 (6.6288)	acc 0.0000 (1.1458)	lr 0.000300
epoch: [1/5][40/404]	time 0.475 (0.495)	data 0.000 (0.011)	eta 0:16:19	loss 6.7128 (6.6098)	acc 0.0000 (1.1719)	lr 0.000300
epoch: [1/5][50/404]	time 0.477 (0.492)	data 0.000 (0.009)	eta 0:16:08	loss 6.4972 (6.5811)	acc 0.0000 (1.3125)	lr 0.000300
epoch: [1/5][60/404]	time 0.477 (0.490)	data 0.003 (0.008)	eta 0:16:00	loss 6.2537 (6.5512)	acc 6.2500 (1.4062)	lr 0.000300
epoch: [1/5][70/404]	time 0.474 (0.489)	data 0.000 (0.007)	eta 0:15:53	loss 6.5482 (6.5208)	acc 3.1250 (2.1429)	lr 0.000300
epoch: [1/5][80/404]	time 0.492 (0.488)	data 0.000 (0.006)	eta 0:15:46	loss 6.2819 (6.4833)	acc 0.0000 (2.3438)	lr 0.000300
epoch: [

#osnet_ain_x0_5

In [27]:
model = torchreid.models.build_model(
    name='osnet_ain_x0_5',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x0_5_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [21]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)
engine.run(
    save_dir='log/osnet_ain_x0_5',
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.245 (0.339)	data 0.000 (0.060)	eta 0:11:20	loss 6.6414 (6.6546)	acc 3.1250 (0.9375)	lr 0.000300
epoch: [1/5][20/404]	time 0.266 (0.300)	data 0.002 (0.030)	eta 0:10:00	loss 6.6431 (6.6674)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/5][30/404]	time 0.251 (0.288)	data 0.001 (0.021)	eta 0:09:32	loss 6.6096 (6.6645)	acc 0.0000 (1.0417)	lr 0.000300
epoch: [1/5][40/404]	time 0.254 (0.280)	data 0.002 (0.016)	eta 0:09:13	loss 6.6140 (6.6497)	acc 0.0000 (1.0156)	lr 0.000300
epoch: [1/5][50/404]	time 0.246 (0.275)	data 0.001 (0.013)	eta 0:09:02	loss 6.6976 (6.6536)	acc 9.3750 (1.5000)	lr 0.000300
epoch: [1/5][60/404]	time 0.251 (0.271)	data 0.000 (0.011)	eta 0:08:51	loss 6.7886 (6.6225)	acc 0.0000 (1.7708)	lr 0.000300
epoch: [1/5][70/404]	time 0.253 (0.268)	data 0.001 (0.009)	eta 0:08:43	loss 6.2140 (6.5653)	acc 0.0000 (2.0982)	lr 0.000300
epoch: [1/5][80/404]	time 0.258 (0.266)	data 0.000 (0.008)	eta 0:08:36	loss 6.0534 (6.5016)	acc 6.2500 (2.6953)	lr 0.000300
epoch: [

#osnet_x0_5

In [25]:
model = torchreid.models.build_model(
    name='osnet_x0_5',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x0_5_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [26]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)
engine.run(
    save_dir='log/osnet_x0_5',
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.283 (0.339)	data 0.000 (0.054)	eta 0:11:21	loss 6.6699 (6.6279)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/5][20/404]	time 0.249 (0.298)	data 0.000 (0.027)	eta 0:09:56	loss 6.6701 (6.6076)	acc 0.0000 (0.4688)	lr 0.000300
epoch: [1/5][30/404]	time 0.245 (0.281)	data 0.001 (0.018)	eta 0:09:19	loss 6.5574 (6.6080)	acc 0.0000 (0.5208)	lr 0.000300
epoch: [1/5][40/404]	time 0.251 (0.273)	data 0.001 (0.014)	eta 0:09:00	loss 6.4137 (6.5930)	acc 0.0000 (0.7031)	lr 0.000300
epoch: [1/5][50/404]	time 0.249 (0.268)	data 0.001 (0.011)	eta 0:08:48	loss 6.4581 (6.5801)	acc 3.1250 (0.8750)	lr 0.000300
epoch: [1/5][60/404]	time 0.255 (0.266)	data 0.000 (0.010)	eta 0:08:40	loss 6.3057 (6.5509)	acc 3.1250 (1.3021)	lr 0.000300
epoch: [1/5][70/404]	time 0.256 (0.264)	data 0.000 (0.008)	eta 0:08:34	loss 6.2200 (6.5024)	acc 12.5000 (1.9643)	lr 0.000300
epoch: [1/5][80/404]	time 0.248 (0.262)	data 0.001 (0.007)	eta 0:08:27	loss 6.1058 (6.4785)	acc 12.5000 (2.7344)	lr 0.000300
epoch: